In [1]:
!qiime tools import \
    --input-path deicode_example/qiita_10422_table.biom \
    --output-path deicode_example/qiita_10422_table.biom.qza \
    --type FeatureTable[Frequency]


Imported deicode_example/qiita_10422_table.biom as BIOMV210DirFmt to deicode_example/qiita_10422_table.biom.qza


In [2]:
!qiime deicode rpca-biplot \
    --i-table deicode_example/qiita_10422_table.biom.qza \
    --p-min-feature-count 10 \
    --p-min-sample-count 500 \
    --o-biplot deicode_example/ordination.qza


Saved PCoAResults % Properties(['biplot']) to: deicode_example/ordination.qza


In [3]:
!qiime emperor biplot \
    --i-biplot deicode_example/ordination.qza \
    --m-sample-metadata-file deicode_example/qiita_10422_metadata.tsv \
    --m-feature-metadata-file deicode_example/taxonomy.qza \
    --o-visualization deicode_example/biplot.qzv \
    --p-number-of-features 8


Saved Visualization to: deicode_example/biplot.qzv


In [8]:
# export for rankviz
!qiime tools export \
    --input-path deicode_example/ordination.qza \
    --output-path deicode_example


Exported deicode_example/ordination.qza as OrdinationDirectoryFormat to directory deicode_example


In [103]:
!rankratioviz_rankplot --help 



Usage: rankratioviz_rankplot [OPTIONS]

Options:
  --ranks TEXT        (str) Path to ordiantion
                      output from deicode & minstel
  --in_biom TEXT      BIOM table describing taxon abundances for samples.
  --in_taxonomy TEXT  Metadata table file for taxonomy.
  --in_metadata TEXT  Metadata table file for samples.
  --in_catagory TEXT  Metadata table catagory to plot.
  --output_dir TEXT   Location of output files.
  --help              Show this message and exit.


In [107]:
!rankratioviz_rankplot \
--ranks deicode_example/ordination.txt \
--in_biom deicode_example/qiita_10422_table.biom \
--in_metadata deicode_example/qiita_10422_metadata.tsv \
--in_taxonomy deicode_example/taxonomy.tsv \
--output_dir deicode_example --in_catagory exposure_type



In [98]:
ordination_res = OrdinationResults.read('deicode_example/ordination.txt')
V = ordination_res.features
U = ordination_res.samples


table = load_table('deicode_example/qiita_10422_table.biom').to_dataframe().to_dense().T
taxam = pd.read_table('deicode_example/taxonomy.tsv')
taxam.set_index('Feature ID',inplace=True)
metadata = pd.read_table('deicode_example/qiita_10422_metadata.tsv')



In [99]:

def matchdf(df1,df2):
    idx = set(df1.index) & set(df2.index)
    return df1.loc[idx],df2.loc[idx]


In [100]:
table,V = matchdf(table.T,V)
table,U = matchdf(table.T,U)

if taxam is not None:
    # match and relabel 
    taxam,V = matchdf(taxam,V)
    if 'Taxon' in taxam.columns \
        and 'Confidence' in taxam.columns:
        #combine and replace
        taxam["Taxon_"] = [str(x)+'_('+str(y)[:4]+')' for x,y in zip(taxam.Taxon,taxam.Confidence)]
        V.index = taxam["Taxon_"].values
        table.columns = taxam["Taxon_"].values
    elif 'Taxon' in taxam.columns:
        #only taxa
        V.index = taxam["Taxon"].values
        table.columns = taxam["Taxon"].values

    

In [102]:

if 'Taxon' in taxam.columns \
    and 'Confidence' in taxam.columns:
    #combine and replace
    taxam["Taxon_"] = [str(x)+'_('+str(y)[:4]+')' for x,y in zip(taxam.Taxon,taxam.Confidence)]
    V.index = taxam["Taxon_"].values
    table.columns = taxam["Taxon_"].values
elif 'Taxon' in taxam.columns:
    #only taxa
    V.index = taxam["Taxon"].values
    table.columns = taxam["Taxon"].values



In [110]:
[(str(x)+'_('+str(y)[:4]+')').replace(' ','') for x,y in zip(taxam.Taxon,taxam.Confidence)]


['k__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Pseudomonadales;f__Moraxellaceae;g__Acinetobacter;s__lwoffii_(0.98)',
 'k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__Ruminococcaceae;g__Ruminococcus;s___(0.98)',
 'k__Bacteria;p__Proteobacteria;c__Alphaproteobacteria;o__Sphingomonadales;f__Sphingomonadaceae;g__Sphingopyxis;s___(0.94)',
 'k__Bacteria;p__Cyanobacteria;c__Chloroplast;o__Streptophyta;f__;g__;s___(0.99)',
 'k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__Lachnospiraceae_(0.80)',
 'k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__;g__;s___(0.72)',
 'k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__;g__;s___(0.95)',
 'k__Bacteria;p__Proteobacteria;c__Alphaproteobacteria;o__Rhizobiales;f__Bradyrhizobiaceae_(0.99)',
 'k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__[Mogibacteriaceae];g__;s___(0.99)',
 'k__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Actinomycetales;f__Corynebacteriaceae;g__Corynebacterium;s_